In [1]:
import os
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path
import torch

@dataclass
class TrainingConfig:
    root_dir: Path
    loss_function: torch.nn.Module
    learning_rate: float
    epochs: int
    device: str
    classes: int
    schedular_params: dict
    bentoml_model_name: str
    train_transform_key: str


In [8]:

#from Food_Classification.config.configuration import ConfigurationManager
from Food_Classification.constants import *
from Food_Classification.utils.common import read_yaml, create_directory
from Food_Classification.entity.config_entity import DataTransformConfig

class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directory([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        
        config = self.config.model_training
        create_directory([config.root_dir])
        
        TrainConfig = TrainingConfig(root_dir= config.root_dir,
                                     loss_function=LOSS_FUNCTION(),
                                     learning_rate= self.params.LEARNING_RATE,
                                     epochs= self.params.EPOCHS,
                                     device= self.params.DEVICE,
                                     classes= self.params.CLASSES,
                                     schedular_params= {"step_size": self.params.STEP_SIZE,
                                                        "gamma": self.params.GAMMA},
                                     bentoml_model_name= "Food_Classification_Model",
                                     train_transform_key= "TRAIN_TRANSFORM_KEY")
        
        return TrainConfig
    
    def get_DataTransformConfig(self) -> DataTransformConfig:
        config = self.config.data_transforms
        create_directory([config.root_dir])
        train_dir = os.path.join(self.config.data_ingestion.unzip_dir,'food_40_percent','train')
        test_dir = os.path.join(self.config.data_ingestion.unzip_dir,'food_40_percent','test')
        train_transformed_file = Path(config.TRAIN_TRANSFORMS_FILE)
        test_transformed_file = Path(config.TEST_TRANSFORMS_FILE)

        TransformationConfig = DataTransformConfig(root_dir= config.root_dir,
                                                   train_dir=Path(train_dir),
                                                   test_dir=Path(test_dir),
                                                    train_transforms_file=train_transformed_file,
                                                    test_transforms_file=test_transformed_file,
                                                   color_transform={"brightness":BRIGHTNESS,
                                                                    "contrast":CONTRAST,
                                                                    "saturation":SATURATION,
                                                                    "hue":HUE},
                                                    spatial_transform= {'vertical_flip': VERTICLE_FLIP,
                                                                        'resize': RESIZE,
                                                                        'center_crop': CENTER_CROP,
                                                                        'rotation': RANDOMROTATION
                                                                        },
                                                    normalize= {'mean': NORMALIZE_MEAN,
                                                                'std': NORMALIZE_STD},
                                                    data_loader_params={'batch_size': self.params.BATCH_SIZE,
                                                                        'shuffle': self.params.SHUFFLE,
                                                                        'num_workers': NUM_WORKERS,
                                                                        'pin_memory': PIN_MEMORY,
                                                                         })
        return TransformationConfig
        



In [11]:
%%writefile src\Food_Classification\components\model_trainer.py
#import torch
#import joblib
#import bentoml
#from Food_Classification.entity.artifact_entity import DataTransformationArtifact, ModelTrainingArtifact
#from tqdm import tqdm
#from torch.optim.lr_scheduler import StepLR, _LRScheduler
#from Food_Classification import logger
#from Food_Classification.config.configuration import ConfigurationManager


class Model_Training:
    def __init__(self, config: TrainingConfig, transformation_Artifacts: DataTransformationArtifact):
        self.config = config
        self.transformation_Artifacts: DataTransformationArtifact = (transformation_Artifacts)
        self.model = torch.load(Path("artifacts/prepare_base_model/base_model_updated.pth"),map_location=torch.device(self.config.device))

    def train_step(self,optimizer: torch.optim.Optimizer):
        '''To train model
        input: model, device, train_dataloader, optimizer, epoch
        output: loss, accuracy'''
        logger.info("Model training train step started")
        try:
            self.model.train()

            train_loss, train_accuracy =0,0
            ProgressBar = tqdm(self.transformation_Artifacts.transformed_train_object)


            for batch, (data, label) in enumerate(ProgressBar):
                data, label = data.to(self.config.device), label.to(self.config.device)

                #forward pass
                y_pred = self.model(data)

                # Calculate the loss 
                loss = self.config.loss_function(y_pred, label)
                train_loss += loss.item()

                #setting optimizer to zero grad
                optimizer.zero_grad()

                # Backward pass
                loss.backward()

                # Update the parameters
                optimizer.step()

                # Calculate the accuracy
                target_predict = torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
                train_accuracy += (target_predict == label).sum().item()/len(y_pred)

            train_loss = train_loss/len(self.transformation_Artifacts.transformed_train_object)
            train_accuracy = train_accuracy/len(self.transformation_Artifacts.transformed_train_object)
            ProgressBar.set_description(f"Train_loss: {train_loss} | Train_accuracy: {train_accuracy}|")

            logger.info("Train step is finished")
              
        except Exception as e:
            raise e

    def test_step(self):
        '''To test model
        input: model, device, test_dataloader
        output: loss, accuracy'''
        logger.info("Model training test step started")

        try:
            self.model.eval()

            test_loss, test_accuracy =0,0
            ProgressBar = tqdm(self.transformation_Artifacts.transformed_test_object)
            with torch.inference_mode():
            
                for batch, (data, label) in enumerate(ProgressBar):
                    data, label = data.to(self.config.device), label.to(self.config.device)

                    label_pred = self.model(data)

                    #calcuate the loss
                    loss = self.config.loss_function(label_pred, label)
                    test_loss += loss.item()

                    # Calculate the accuracy
                    target_predict = torch.argmax(torch.softmax(label_pred,dim=1),dim=1)
                    test_accuracy += (target_predict == label).sum().item()/len(label_pred)

            test_loss = test_loss/len(self.transformation_Artifacts.transformed_test_object)
            test_accuracy = test_accuracy/len(self.transformation_Artifacts.transformed_test_object)
            logger.info("Test step finished")
        except Exception as e:
            raise e


    def initiate_Model_training(self) -> ModelTrainingArtifact:
        try:
            logger.info("Model training started")
            model: torch.nn.Module = self.model.to(self.config.device)
            optimizer: torch.optim.Optimizer = torch.optim.Adam(model.parameters(),lr = self.config.learning_rate)
            schedular: _LRScheduler = StepLR(optimizer=optimizer,
                                             **self.config.schedular_params)
            
            for epoch in range(1,self.config.epochs+1):
                logger.info(f"Epoch: {epoch}")
                self.train_step(optimizer=optimizer)
                optimizer.step()
                schedular.step()
                self.test_step()

            
            trained_model_path = os.path.join(self.config.root_dir, "trained_model.pth")
            torch.save(model, trained_model_path)

            train_trans_obj  = joblib.load(self.transformation_Artifacts.trained_transformed_file)
            bentoml.pytorch.save_model(name= self.config.bentoml_model_name,
                                       model=model,
                                       custom_objects={self.config.train_transform_key: train_trans_obj}
                                       )
            
            
            model_training_artifact: ModelTrainingArtifact = ModelTrainingArtifact(
                trained_model_path= trained_model_path)
            
            logger.info("MOdel training Complete")
            return model_training_artifact

        except Exception as e:
            raise e


Overwriting src\Food_Classification\components\model_trainer.py


In [10]:
from Food_Classification.entity.config_entity import DataTransformConfig
from Food_Classification.components.data_transformation import DataTransformation
#from Food_Classification.config.configuration import ConfigurationManager
config = ConfigurationManager()
tran = config.get_DataTransformConfig()
transfom = DataTransformation(tran)
transfom_artifact = transfom.initiate_datatransfrom()
training_config = config.get_training_config()
training = Model_Training(config=training_config, transformation_Artifacts= transfom_artifact)
training.initiate_Model_training()

[2024-05-20 14:20:30,944: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-05-20 14:20:30,946: INFO: common: yaml file params.yaml loaded successfully]
[2024-05-20 14:20:30,947: INFO: common: directory artifacts created successfully]
[2024-05-20 14:20:30,948: INFO: common: directory artifacts/data_transforms created successfully]
[2024-05-20 14:20:30,949: INFO: data_transformation: Data Transformation started]
[2024-05-20 14:20:30,953: INFO: data_transformation: transform pickle file created]
[2024-05-20 14:20:30,954: INFO: data_transformation: Data Loading started]
[2024-05-20 14:20:30,994: INFO: data_transformation: Data Transformation completed]
[2024-05-20 14:20:31,016: INFO: common: directory artifacts/model_training created successfully]
[2024-05-20 14:20:31,222: INFO: 3185675670: Model training started]
[2024-05-20 14:20:31,225: INFO: 3185675670: Epoch: 1]
[2024-05-20 14:20:31,226: INFO: 3185675670: Model training train step started]


100%|██████████| 188/188 [00:55<00:00,  3.39it/s]

[2024-05-20 14:21:26,664: INFO: 3185675670: Train step is finished]
[2024-05-20 14:21:26,665: INFO: 3185675670: Model training test step started]



100%|██████████| 63/63 [00:39<00:00,  1.60it/s]

[2024-05-20 14:22:05,978: INFO: 3185675670: Test step finished]
[2024-05-20 14:22:06,128: INFO: pytorch: Using the default model signature for PyTorch ({'__call__': {'batchable': False}}) for model "Food_Classification_Model".]
[2024-05-20 14:22:06,128: WARNING: tag: Converting 'Food_Classification_Model' to lowercase: 'food_classification_model'.]


[2024-05-20 14:22:07,010: INFO: _client: HTTP Request: POST https://t.bentoml.com "HTTP/1.1 200 OK"]
[2024-05-20 14:22:07,011: INFO: 3185675670: MOdel training Complete]


ModelTrainingArtifact(trained_model_path='artifacts/model_training\\trained_model.pth')